## Notebook contém código para extrair e combinar df de diferentes arquivos

In [1]:
import os
import numpy as np
import pandas as pd
import unicodedata, re
import matplotlib.pyplot as plt
#from jupyter_core.migrate import regex

In [2]:
def load_data(name):
    tmp = pd.read_csv(f"../data/{name}")
    tmp.rename(columns={'Name': 'name',
                           'Price': 'price',
                           'Brand': 'brand',
                           'Category': 'category'}, inplace=True)
    tmp = tmp[['name', 'price', 'category', 'brand']]
    tmp.dropna(inplace=True)

    return tmp

In [3]:
files = ["df_nondim.csv", "df.csv", "tupan_0x0.csv",
         "tupan_0x1.csv", "tupan_0x2.csv", "tupan_0x3.csv"]
dt = pd.DataFrame(columns=['name', 'price', 'category', 'brand'])
for file in files:
    aux = load_data(file)
    dt = pd.concat([dt.astype(aux.dtypes), aux.astype(dt.dtypes)],
                   ignore_index=True)
dt.drop_duplicates(subset=["name"], inplace=True)
raw_df = dt.copy(deep=True)
print(f"shape: {dt.shape}")

shape: (13060, 4)


In [11]:
dt.to_csv("clean_dt.csv", index=False)

In [4]:
def handle_name_0x1(row):
    name = unicodedata.normalize('NFD', str(row))

    name = re.sub(r',', '.', name)
    name = re.sub(r'[></-]', '', name)
    name = re.sub(r'\s*/?\s*ref\.\s*\w+', '', name, flags=re.IGNORECASE)

    name = ''.join([char for char in name if not unicodedata.combining(char)])
    return name
def handle_cate_0x1(row):
    category = unicodedata.normalize('NFD', str(row))
    category = category.lower()
    category = re.sub(r'[0-9]', '', category)
    category = ''.join([
        char for char in category
        if not unicodedata.combining(char)])
    words = category.split()
    unique_words = []
    [unique_words.append(word)
     for word in words if word not in unique_words]
    unique_category = " ".join(unique_words)
    #category = unique_category.upper()
    return category
def handle_name_category(row):
    name = unicodedata.normalize('NFD', str(row['name']))
    category = unicodedata.normalize('NFD', str(row['category']))
    #name = name.lower()
    category = category.lower()

    name = re.sub(r',', '.', name)
    name = re.sub(r'[></-]', '', name)
    category = re.sub(r'[0-9.,>/-]', '', category)
    name = re.sub(r'\s*/?\s*ref\.\s*\w+', '', name, flags=re.IGNORECASE)

    name = ''.join([char for char in name if not unicodedata.combining(char)])
    category = ''.join([char for char in category if not unicodedata.combining(char)])

    words = category.split()
    unique_words = []
    [unique_words.append(word)
      for word in words if word not in unique_words]
    unique_category = " ".join(unique_words)
    row["name"] = name
    row["category"] = unique_category.upper()
    row["features"] = row["name"] + " SEP " + unique_category
    return row

#df = df.apply(handle_name_category, axis=1)
#df = df.drop_duplicates(subset=["name"])
#print(f"shape: {df.shape}")
#df["name"] = df["name"].apply(handle_name_0x1)
dt["category"] = dt["category"].apply(handle_cate_0x1)
df = dt.copy()
dt.head(5)

,name,price,category,brand
0,Telha Ecológica Clássica Fit 200x75cm Vermelho...,85.9,telhas > telhas de fibra vegetal,ONDULINE
1,Pneu 325 Aro 8 com 2 Lonas Leve COLSON / REF. ...,32.9,maquinas para construcao > carro de mao,COLSON
2,Caixa para Massa de Plástico 20 Litros Reforça...,24.9,acessorios para construcao > acessorios para c...,DIMAX BR
3,"Esquadro em Aço 14 Polegadas x35,5cm com Cabo ...",18.9,acessorios para construcao > acessorios para c...,DIMAX BR
4,"Esquadro em Aço 12 Polegadas x30,4cm com Cabo ...",17.9,acessorios para construcao > acessorios para c...,DIMAX BR


In [ ]:
values_0x0 = {"acabamentos":"acabamentos",  "acessorios agricolas":"acess_agricolas", "acessorios automotivos":"acess_automotivos",
              "acessorios de iluminacao de uso geral":"acess_iluminacao",
              "acessorios de jardinagem":"acess_jardinagem", "jardim":"acess_jardinagem", "maquinas e ferramentas de jardinagem":"acess_jardinagem",
              "acessorios de tecnologia":"acess_tecnologia","seguranca":"acess_tecnologia",
              "acessorios e conexoes eletricas":"acess_conexoes_eletricas", "fitas e mangueiras de led":"acess_conexoes_eletricas",
              "kits trilho":"acess_conexoes_eletricas", "acessorios e utensilios de cozinha":"acess_utensilios_cozinha",
              "acessorios e utensilios de lavanderia":"acess_utensilios_lavanderia", "acessorios para construcao":"acess_construcao",
              "ferramentas para construcao":"acess_construcao", "maquinas para construcao":"acess_construcao",
              "acessorios para decoracao":"acess_decoracao", "acessorios para ferramentas":"acess_ferramentas", "acessorios para pintura":"acess_pintura",
              "acessorios para pisos e revestimentos":"acess_pisos_revestimentos", "acessorios sanitarios":"acess_sanitarios", "aco":"aco",
              "adornos":"adornos", "outros adornos":"adornos", "arandelas":"arandelas", "argamassas e rejuntes":"argamassas_rejuntes",
              "massa":"argamassas_rejuntes", "materiais de campo":"argamassas_rejuntes",
              "assentos sanitarios":"assentos_sanitarios", "bacias e caixas":"bacias_caixas", "banho":"banho", "bombas e filtros":"bombas_filtros",
              "brocas":"brocas", "cabos, fios e conectores":"cabos_fios_conectores", "caixas, quadros e disjuntores":"caixas_quadros_disjuntores",
              "cama":"cama", "casa":"casa", "churrasco":"churrasco", "chuveiros":"chuveiros", "complementos":"complementos", "conexoes":"conexoes",
              "cortinas e persianas":"cortinas_persianas", "cozinha":"cozinha", "cubas, pias e balcoes":"cubas_pias_balcoes",
              "discos e rebolos":"discos_rebolos", "duchas higienicas":"duchas_higienicas", "eletroportateis":"eletroportateis", "epi":"epi",
              "fechaduras":"fechaduras",
              "ferramentas a bateria":"ferramentas_eletricas", "ferramentas eletricas":"ferramentas_eletricas",
              "ferramentas agricolas":"ferramentas agricolas",
              "ferramentas manuais":"ferramentas_manuais",
              "impermeabilizantes":"impermeabilizantes", "interruptores":"interruptores_tomadas", "tomadas":"interruptores_tomadas",
              "janelas":"janelas", "lampadas":"lampadas", "lavatorios":"lavatorios", "limpeza":"limpeza", "lonas":"lonas",
              "luminarias":"luminarias_refletores", "refletores":"luminarias_refletores", "spots":"luminarias_refletores",
              "moveis":"moveis", "paineis de led":"paineis_led", "pinceis":"pinceis", "pisos":"pisos", "porcelanatos":"porcelanatos", "portas":"portas",
              "prateleiras":"prateleiras", "registros":"registros", "revestimentos":"revestimentos", "tintas":"tintas", "torneiras":"torneiras",
              "piscinas":"piscinas_tratamento","tratamento":"piscinas_tratamento", "tubos":"tubos", "ferragens":"ferragens"
              }
keys_0x1 = {("grelhas", "calhas"): "acess_hidraulico", "acessorios hidraulicos de uso geral": "acess_hidraul_geral"}
len(values_0x0)

In [5]:
#startswith
keys_0x0 = ["acabamentos", "acessorios agricolas","acessorios automotivos", "acessorios de iluminacao de uso geral",
        ("acessorios de jardinagem", "jardim", "maquinas e ferramentas de jardinagem"), ("acessorios de tecnologia","seguranca"), ("acessorios e conexoes eletricas", "fitas e mangueiras de led", "kits trilho"),
        "acessorios e utensilios de cozinha", "acessorios e utensilios de lavanderia", ("acessorios para construcao", "ferramentas para construcao", "maquinas para construcao"),
        "acessorios para decoracao", "acessorios para ferramentas", "acessorios para pintura",
        "acessorios para pisos e revestimentos", "acessorios sanitarios", "aco", ("adornos", "outros adornos"), "arandelas",
        ("argamassas e rejuntes", "massa", "materiais de campo"), "assentos sanitarios", "bacias e caixas", "banho", "bombas e filtros",
        "brocas", "cabos, fios e conectores", "caixas, quadros e disjuntores", "cama", "casa", "churrasco",
        "chuveiros", "complementos", "conexoes", "cortinas e persianas", "cozinha", "cubas, pias e balcoes",
        "discos e rebolos", "duchas higienicas", "eletroportateis", "epi", "fechaduras",
        ("ferramentas a bateria", "ferramentas eletricas"), "ferramentas agricolas", "ferramentas manuais",
        "impermeabilizantes", ("interruptores", "tomadas"), "janelas", "lampadas", "lavatorios", "limpeza", "lonas",
        ("luminarias", "refletores", "spots"), "moveis", "paineis de led", "pinceis", "pisos", "porcelanatos", "portas", "prateleiras", "registros", "revestimentos", "tintas", "torneiras", ("piscinas","tratamento"),
        "tubos",
        ]
#endswith
keys_0x1 = [("grelhas", "calhas"):"acess_hidraulico", "acessorios hidraulicos de uso geral":"acess_hidraul_geral"]

In [6]:
len(keys_0x0)

64

In [13]:
dt[dt.category.str.startswith("tintas")][["name","category"]]
#dt[dt.category.str.endswith(("cadeados"))][["name", "category"]]

,name,category
1887,Tinta Esmalte Auto Brilho 750ml Dialine Base Á...,tintas esmaltes > esmaltes premium
1888,Tinta Esmalte Auto Brilho 750ml Dialine Base Á...,tintas esmaltes > esmaltes premium
1889,Tinta Esmalte Auto Brilho 750ml Dialine Base Á...,tintas esmaltes > esmaltes premium
1890,Tinta Esmalte Auto Brilho 750ml Dialine Base Á...,tintas esmaltes > esmaltes premium
1891,Tinta Esmalte Auto Brilho 750ml Dialine Base Á...,tintas esmaltes > esmaltes premium
...,...,...
31740,Tinta Esmalte Sintético Brilhante Hammerite Pr...,tintas esmaltes > esmaltes premium
31741,Tinta Esmalte Sintético Brilhante Hammerite Pr...,tintas esmaltes > esmaltes premium
31742,Tinta Esmalte Sintético Brilhante Hammerite Pr...,tintas esmaltes > esmaltes premium
31743,Tinta Esmalte Sintético Brilhante Hammerite Pr...,tintas esmaltes > esmaltes premium


In [39]:
#df[df.category.str.startswith("portas")] x
# df[(df["category"].str.startswith("ferragens")
#    | df["category"].str.endswith("ferragens"))
#    & ~df["category"].str.endswith(("fechaduras", "cadeados"))] #= ferragens ok
# df[df["category"].str.endswith(("parafusos", "buchas"))]
# df[df.category.str.startswith(("cabos", "fios", "conectores"))] #= cabos_fios ok
# df[df.category.str.startswith("acessorios sanitarios") &
#    ~df.category.str.endswith("acessorios avulsos")] #= acessorio_sanitario ok
#df[df.category.str.endswith("acessorios avulsos")] #= acessorios_sani_avulsos
#df[df.category.str.startswith(("caixas", "quadros" "disjuntores"))] #= caixas_disjuntores ok
# df[df.category.str.endswith(
#     ("tomadas", "modulares", "tomadas simples", "tomadas conjunto", "interruptores", "interruptores simples", "interruptores conjunto"))] #tomadas_interruptores #= ok
# df[df.category.str.endswith(
#     ("sensores de presenca", "molduras e placas", "sistema x", "campanhias"))] #= sensores_placas ok
#df[df.category.str.startswith("ferramentas manuais")] #= ferramentas_manuais ok x
#df[df.category.str.startswith("ferramentas eletricas")] #= ferramentas_eletricas ok
#df[df.category.str.startswith("ferramentas a bateria")]#= ferramentas_eletricas ok
#df[df.category.str.startswith("acessorios para ferramentas")] #= acessorios_ferramenta ok x
#df[df.category.str.startswith("tintas")] #= tintas ok x
#df[df.category.str.startswith(("complementos", "seladores"))] #= complementos_tintas ok
#df[df.category.str.startswith(("pinceis", "rolos"))] #= acessorios_pintura ok
#df[df.category.str.startswith(("porcelanatos"))] #= porcelanato ok x
#df[df.category.str.startswith(("pisos"))] #=pisos ok x
#df[df.category.str.startswith(("revestimentos"))] #= revestimentos ok x
#df[df.category.str.startswith("acessorios para pisos")] #= aces_pisos ok x
#df[df.category.str.startswith("tubos") & ~df.category.str.endswith("conexoes")] #= tubos ok
#df[df.category.str.startswith("conexoes") | df.category.str.endswith("conexoes")] #= conexoes ok
#df[df.category.str.startswith("acessorios hidraulicos")] #= acessorios_hidraulicos ok x
#df[df.category.str.startswith("reservatorios")] #= reservatorios ok uns x
#df[df.category.str.startswith("bombas")] #= bombas_filtros ok x
#df[df.category.str.startswith("telhas")] #= telhas ok x
#df[df.category.str.startswith(("materiais de campo", "cimento"))] #= materiais_campo ok insuf
#df[df.category.str.startswith(("aco", "complementos de aco"))] #= aco ok insuf
#df[df.category.str.startswith("lonas")] #= lonas ok insuf x
#df[df.category.str.startswith("acabamentos")] #= acabamentos x
#df[df.category.str.startswith(("cubas", "pias", "balcoes"))] #= cubas_pias
# df[df.category.str.startswith("fechaduras")
# | df["category"].str.endswith("fechaduras")][["name", "category"]] #= fechaduras
#df[df.category.str.startswith("ferramentas para construcao")] #= ferramentas_contrucao x

In [45]:
patterns = {
    "portas": "portas", "ferramentas manuais": "ferramentas_manuais",
    "acessorios para ferramentas": "acessorios_ferramentas", "tintas": "tintas",
    "porcelanatos": "porcelanatos", "pisos": "pisos", "revestimentos": "revestimentos",
    "ferramentas para construcao": "ferramentas_contrucao", "acabamentos": "acabamentos",
    "lonas": "lonas", "telhas": "telhas", "bombas": "bombas_filtros", "reservatorios": "reservatorios",
    "acessorios hidraulicos": "acessorios_hidraulicos", "acessorios para pisos": "acessorios_pisos"
 }

In [54]:
for pattern, new_category in patterns.items():
    df["category"].loc[df['category'].str.startswith(pattern)] = new_category

In [59]:
df[df.category.str.startswith())]

,name,price,category,brand
0,Telha Ecológica Clássica Fit 200x75cm Vermelho...,85.9,telhas,ONDULINE
5,Telha Ecológica Stilo 3D 96cmx2Metros Vermelho...,109.9,telhas,ONDULINE
11,Kit Parafuso 10 Peças para Fixação de Telha Co...,16.9,telhas,PRECONVC
12,Telha Ecológica Stilo 3D 96cmx2Metros Verde ON...,99.9,telhas,ONDULINE
13,Telha Ecológica Clássica Fit 75cmx2Metros Verd...,89.9,telhas,ONDULINE
...,...,...,...,...
54003,Porta de Giro Basculante Lado Direito Home de ...,519.9,portas,QUALITY
54268,Chave Teste Com Ponta Fenda 1/8X3 Polegadas 80...,20.9,ferramentas_manuais,TRAMONTINA
54300,Arco Serra Fixo em Aço 12 Polegadas Com Cabo P...,35.9,ferramentas_manuais,TRAMONTINA
54301,Arco de Serra em Aço 12 Polegadas Fixo Eco com...,25.9,ferramentas_manuais,DIMAX
